In [38]:
import cv2
import numpy as np
import time

The following function takes two images, and if both contain a 9*6 chessboard, it saves them.

In [39]:
# left: left image
# right: right image
# i: number of pairs that have been saved until now
# returns: True if image pair should be saved, False otherwise
def saveAndDrawIfChessboardCornersFound(left,right,i):
    leftFound, leftCorners = cv2.findChessboardCorners(left,(9,6))
    # Save some computation time, only search second image if first
    # contained a chessboard
    if leftFound:
        rightFound, rightCorners = cv2.findChessboardCorners(right,(9,6))
        if rightFound:
            cv2.imwrite("left_"+str(i)+".png",left)
            cv2.imwrite("right_"+str(i)+".png",right)
            leftCopy=left.copy()
            rightCopy=right.copy()
            cv2.drawChessboardCorners(leftCopy,(9,6),leftCorners,leftFound)
            cv2.drawChessboardCorners(rightCopy,(9,6),rightCorners,rightFound)
            cv2.imshow("Left Chessboard corners",leftCopy)
            cv2.moveWindow("Left Chessboard corners",-150,0)
            cv2.imshow("Right Chessboard corners",rightCopy)
            cv2.moveWindow("Right Chessboard corners",150,0)
            return True
    return False

We iterate over two webcams until we have saved enough (nbChessboardPairsWanted) pairs of images containing a chessboard.

In [40]:
capLeft=cv2.VideoCapture(1)
capRight=cv2.VideoCapture(0)

nbPairsAcquired=0
nbChessboardPairsFound=0
nbChessboardPairsWanted=50
skip=10

while nbChessboardPairsFound<nbChessboardPairsWanted:
    try:
        retLeft,left=capLeft.read()
        retRight,right=capRight.read()
        
        if retLeft and retRight:
            cv2.imshow("Left image",left)
            cv2.imshow("Right image",right)
            cv2.moveWindow("Left image",0,200)
            cv2.moveWindow("Right image",900,200)
            nbPairsAcquired+=1
            if nbPairsAcquired%skip==0:
                if saveAndDrawIfChessboardCornersFound(left,right,nbChessboardPairsFound):
                    nbChessboardPairsFound+=1
                    
        else:
            print("Couldn't grab images from both cameras:")
            print("\t - left: "+str(retLeft))
            print("\t - right: "+str(retRight))
            break
            
        # 113 is the ASCII code for 'q'
        if cv2.waitKey(10) == 113:
            break
    except:
        print("Something went wrong!")
        time.sleep(1)
        capLeft.release()
        capRight.release()
        break
        
        
# Do not forget to release the cameras...
capLeft.release()
capRight.release()

# ...and close all windows
cv2.destroyAllWindows()